# Market Analysis: S&P 500 and Sector Performance

Interactive dashboard for S&P 500 and major sector ETFs with selectable time ranges (1w, 1m, 3m, All), metric toggles (Price / Normalized / Cumulative %), and rolling volatility.

In [35]:
# 1. Load Dependencies (MongoDB only data source)
import os
from datetime import datetime, timedelta, date
from typing import Dict
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pymongo import MongoClient
import yaml

try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

MONGO_URI = os.getenv('MONGO_URI', 'mongodb://localhost:27017')
DB_NAME = os.getenv('DB_NAME', 'FIN')
STOCK_COLLECTION = os.getenv('STOCK_COLLECTION', 'stock_daily')
INDEX_COLLECTION = os.getenv('INDEX_COLLECTION', 'index_daily')


In [39]:
# 2. Configuration
with open('directory.yml') as f:
    directory = yaml.safe_load(f)
NAME_MAP : Dict[str, str] = directory.get('indexes', {})
TODAY = date.today()
LOOKBACK_DAYS = int(os.getenv('LOOKBACK_DAYS','180'))
END_DATE = datetime.strptime(os.getenv('END_DATE', TODAY.strftime('%Y-%m-%d')), '%Y-%m-%d').date()
START_DATE = datetime.strptime(os.getenv('START_DATE', (END_DATE - timedelta(days=LOOKBACK_DAYS)).strftime('%Y-%m-%d')), '%Y-%m-%d').date()
ALL_TICKERS = list(NAME_MAP.keys())
print(f"DB tickers: {ALL_TICKERS}\nRange: {START_DATE} -> {END_DATE}")

DB tickers: ['^SPX', '^SP500-50', '^SP500-25', '^SP500-30', '^GSPE', '^SP500-40', '^SP500-35', '^SP500-20', '^SP500-45', '^SP500-15', '^SP500-60', '^SP500-55']
Range: 2025-04-01 -> 2025-09-28


In [3]:
# 3. Load Data From MongoDB (no external API)
client = MongoClient(MONGO_URI)
db = client[DB_NAME]
coll = db[INDEX_COLLECTION]

query = {
    'ticker': {'$in': ALL_TICKERS},
    'date': {'$gte': datetime(START_DATE.year, START_DATE.month, START_DATE.day),
             '$lte': datetime(END_DATE.year, END_DATE.month, END_DATE.day)}
}
projection = {'_id': 0, 'ticker': 1, 'date': 1, 'close': 1}

rows = list(coll.find(query, projection))
if not rows:
    raise ValueError('No data returned from MongoDB for specified range. Ensure update script has populated index_daily collection.')

df = pd.DataFrame(rows)
# Ensure datetime & pivot
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True)
prices = df.pivot(index='date', columns='ticker', values='close')
prices = prices.reindex(columns=ALL_TICKERS)  # consistent order
print(prices.tail())

ticker             ^SPX   ^SP500-50    ^SP500-25   ^SP500-30  ^SP500-10TR  \
date                                                                        
2025-09-22  6693.750000  434.510010  1933.280029  869.559998          NaN   
2025-09-23  6656.919922  432.260010  1905.479980  872.659973          NaN   
2025-09-24  6637.970215  428.709991  1918.770020  873.830017          NaN   
2025-09-25  6604.720215  425.970001  1890.660034  868.700012          NaN   
2025-09-26  6643.700195  426.529999  1918.050049  867.750000  1655.630005   

ticker       ^SP500-40    ^SP500-35    ^SP500-20    ^SP500-45   ^SP500-15  \
date                                                                        
2025-09-22  898.770020  1593.800049  1295.500000  5616.259766  576.840027   
2025-09-23  894.789978  1597.630005  1295.680054  5552.470215  575.289978   
2025-09-24  892.719971  1591.869995  1289.119995  5523.810059  566.150024   
2025-09-25  890.299988  1565.229980  1280.619995  5525.580078  559.309998  

In [4]:
# 4-6. Transformations
if prices.isna().all().all():
    raise ValueError('All price values are NaN; check DB contents.')
normalized = prices / prices.iloc[0] * 100
returns = prices.pct_change().fillna(0)
cumulative = (1 + returns).cumprod() - 1
rolling_vol = returns.rolling(20).std() * np.sqrt(252)
DATA = {'price': prices, 'normalized': normalized, 'cumulative': cumulative}
print('Transformed frames ready.')

Transformed frames ready.


In [40]:
# 7-11. Plot with Precomputed Window Rebasing (1w / 1m / 3m)
import plotly.io as pio
from dateutil.relativedelta import relativedelta

# Precompute window slices & rebased datasets
last_date = prices.index.max()
if pd.isna(last_date):
    raise ValueError('No last_date found in prices index')

WINDOW_SPECS = {
    '1w': last_date - timedelta(days=7),
    '1m': last_date - relativedelta(months=1),
    '3m': last_date - relativedelta(months=3),
}

# Ensure start boundaries not before available data
first_date = prices.index.min()
rebased_sets = {}
rebased_vol = {}
for label, start_bound in WINDOW_SPECS.items():
    start_bound = max(first_date, start_bound)
    window_prices = prices.loc[start_bound:last_date]
    base = window_prices.iloc[0]
    rb_norm = window_prices / base * 100
    rb_returns = window_prices.pct_change().fillna(0)
    rb_cum = (1 + rb_returns).cumprod() - 1
    rb_vol = rb_returns.rolling(20).std() * np.sqrt(252)
    rebased_sets[label] = {
        'price': window_prices,
        'normalized': rb_norm,
        'cumulative': rb_cum,
    }
    rebased_vol[label] = rb_vol

DEFAULT_WINDOW = '3m'

# Figure builder using a selected window dataset
def make_window_figure(window_key: str):
    data = rebased_sets[window_key]
    vol = rebased_vol[window_key]
    norm = data['normalized']
    fig_local = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.72,0.28], vertical_spacing=0.05,
                              subplot_titles=(f"Performance (Base=100) – {window_key}", "Rolling 20D Vol (Ann.)"))
    for col in norm.columns:
        fig_local.add_trace(go.Scatter(x=norm.index, y=norm[col], name=NAME_MAP.get(col,col), mode='lines'), row=1, col=1)
    for col in vol.columns:
        fig_local.add_trace(go.Scatter(x=vol.index, y=vol[col], name=f"Vol {NAME_MAP.get(col,col)}", mode='lines', line=dict(dash='dot'), opacity=0.55, showlegend=False), row=2, col=1)

    # Linear axes enforcement
    fig_local.update_yaxes(type='linear', title_text='Index (Base=100)', row=1, col=1)
    fig_local.update_yaxes(type='linear', title_text='Ann Vol', row=2, col=1)

    # We remove range buttons; keep simple slider
    fig_local.update_xaxes(rangeslider=dict(visible=True), row=2, col=1)

    dropdown_buttons = [
        dict(label='Normalized', method='update', args=[{'y': [data['normalized'][c] if i < len(data['normalized'].columns) else vol[c.replace('Vol ','')] if c.startswith('Vol') else [] for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]}, {'yaxis': {'title':'Index (Base=100)', 'type':'linear'}}]),
        dict(label='Price', method='update', args=[{'y': [data['price'][c] if i < len(data['price'].columns) else vol[c.replace('Vol ','')] if c.startswith('Vol') else [] for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]}, {'yaxis': {'title':'Price', 'type':'linear'}}]),
        dict(label='Cumulative %', method='update', args=[{'y': [(data['cumulative'][c]*100) if i < len(data['cumulative'].columns) else vol[c.replace('Vol ','')] if c.startswith('Vol') else [] for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]}, {'yaxis': {'title':'Cumulative %', 'type':'linear'}}]),
    ]

    fig_local.update_layout(
        updatemenus=[
            dict(type='dropdown', x=1.02, y=1.0, xanchor='left', buttons=dropdown_buttons, showactive=True),
            dict(type='dropdown', x=1.02, y=0.85, xanchor='left', showactive=True,
                 buttons=[
                     dict(label='1w', method='update', args=[{'x': [rebased_sets['1w']['normalized'].index for _ in norm.columns + list(vol.columns)],
                                                              'y': [rebased_sets['1w']['normalized'][c] if i < len(rebased_sets['1w']['normalized'].columns) else rebased_vol['1w'][c.replace('Vol ','')] if c.startswith('Vol') else []
                                                                     for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]},
                                                             {'title': 'Performance (Base=100) – 1w', 'yaxis': {'type':'linear'}}]),
                     dict(label='1m', method='update', args=[{'x': [rebased_sets['1m']['normalized'].index for _ in norm.columns + list(vol.columns)],
                                                              'y': [rebased_sets['1m']['normalized'][c] if i < len(rebased_sets['1m']['normalized'].columns) else rebased_vol['1m'][c.replace('Vol ','')] if c.startswith('Vol') else []
                                                                     for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]},
                                                             {'title': 'Performance (Base=100) – 1m', 'yaxis': {'type':'linear'}}]),
                     dict(label='3m', method='update', args=[{'x': [rebased_sets['3m']['normalized'].index for _ in norm.columns + list(vol.columns)],
                                                              'y': [rebased_sets['3m']['normalized'][c] if i < len(rebased_sets['3m']['normalized'].columns) else rebased_vol['3m'][c.replace('Vol ','')] if c.startswith('Vol') else []
                                                                     for i,c in enumerate([*norm.columns, *[f'Vol {v}' for v in vol.columns]])]},
                                                             {'title': 'Performance (Base=100) – 3m', 'yaxis': {'type':'linear'}}]),
                 ])
        ],
        template='plotly_dark', hovermode='x unified', legend=dict(title='Index / Sector')
    )

    return fig_local

fig = make_window_figure(DEFAULT_WINDOW)
fig.show()

## Weekly Performance Extensions

This section adds:
1. Top 5 stock range movers (weekly high - weekly low) from the configured stock list.
2. Markdown table of S&P 500 and sector weekly & YTD returns.
3. Mermaid radar showing weekly High / Low / Close percentages vs prior week close.

In [41]:
# Helper to get last completed week (Mon-Fri) bounds
import pandas as pd
from datetime import timedelta

# Using index/sector collection for SP500 & sectors; for individual stocks pull from stock_daily
stock_coll = db[STOCK_COLLECTION]
index_coll = db[INDEX_COLLECTION]

# Determine last trading week using index data dates available
all_dates = prices.index
last_day = all_dates.max().date()
# Assume last week ends at last_day (could refine to last Friday if run on weekend)
end_week = last_day
start_week = end_week - timedelta(days=6)

print(f"Weekly window: {start_week} -> {end_week}")


Weekly window: 2025-09-20 -> 2025-09-26


In [42]:
with open('directory.yml') as f:
    a = yaml.safe_load(f)
    #a = list(a)
print(a)

{'stocks': {'meme': {'description': 'MEME stocks', 'tickers': ['GME', 'AMC', 'OPEN', 'KSS', 'DNUT', 'UUUU', 'BB', 'FFAI', 'GLXY', 'PEW', 'BBAI', 'GPRO', 'BYND', 'RKT', 'NVAX']}, 'warfare': {'description': 'Defense stocks', 'tickers': ['LMT', 'RTX', 'NOC', 'BA', 'EUAD']}, 'daily_life_products': {'description': 'Favorite products', 'tickers': ['RDDT', 'GTLB', 'ZG', 'GOOGL', 'COF', 'KR', 'H', 'JD', 'PEP', 'CATO', 'HOOD', 'HPE']}, 'homie': {'description': 'Homie stocks for option trading', 'tickers': ['OPEN', 'QBTS', 'MP', 'TSLA', 'UNH', 'NVDA', 'AMD', 'INTC', 'ADBE', 'PLTR', 'ORCL', 'GTLB']}}, 'indexes': {'^SPX': 'S&P 500', '^SP500-50': 'Comm Services', '^SP500-25': 'Cons Disc', '^SP500-30': 'Cons Staples', '^GSPE': 'Energy', '^SP500-40': 'Financials', '^SP500-35': 'Health', '^SP500-20': 'Industrials', '^SP500-45': 'Tech', '^SP500-15': 'Materials', '^SP500-60': 'Real Estate', '^SP500-55': 'Utilities'}}


In [44]:
# 1. Top 5 stock range movers (High - Low within week) using stock.yml
import yaml
from pathlib import Path

# Load tickers from stock.yml multi-document structure
stock_yml_path = 'directory.yml'
interest_set = set()
with open(stock_yml_path, 'r', encoding='utf-8') as f:
    stock_groups = yaml.safe_load(f).get('stocks', []) # dict

for name, group in stock_groups.items():
    tickers = group.get('tickers', [])
    interest_set.update(tickers)

if not interest_set:
    print('No interest tickers found in stock.yml')

start_dt = datetime(start_week.year, start_week.month, start_week.day)
end_dt = datetime(end_week.year, end_week.month, end_week.day)
stock_query = {
    'ticker': {'$in': sorted(list(interest_set))},
    'date': {'$gte': start_dt, '$lte': end_dt}
}
stock_proj = {'_id':0,'ticker':1,'date':1,'high':1,'low':1,'close':1}
stock_rows = list(stock_coll.find(stock_query, stock_proj))
stock_df = pd.DataFrame(stock_rows)
if stock_df.empty:
    print('No stock data for interest list in week window.')
else:
    stock_df['date'] = pd.to_datetime(stock_df['date'])
    agg = stock_df.groupby('ticker').agg(week_high=('high','max'), week_low=('low','min'))
    agg['range'] = agg['week_high'] - agg['week_low']
    top_movers = agg.sort_values('range', ascending=False).head(10)
    display(top_movers)

,week_high,week_low,range
ticker,,,
ORCL,329.50,283.000,46.500
RDDT,262.99,223.500,39.490
TSLA,444.98,419.080,25.900
NOC,594.96,570.260,24.700
LMT,495.24,473.010,22.230
UNH,354.50,332.600,21.900
ADBE,368.73,347.800,20.930
GOOGL,255.78,240.740,15.040
MP,82.23,68.570,13.660


In [45]:
# 2. Weekly & YTD returns table for SP500 + sectors
# Fetch required period: YTD start
YTD_START = date(end_week.year, 1, 1)
ytd_query = {
    'ticker': {'$in': ALL_TICKERS},
    'date': {'$gte': datetime(YTD_START.year,1,1), '$lte': end_dt}
}
idx_proj = {'_id':0,'ticker':1,'date':1,'close':1,'high':1,'low':1}
ytd_rows = list(index_coll.find(ytd_query, idx_proj))
idx_df = pd.DataFrame(ytd_rows)
if idx_df.empty:
    print('No index data for YTD calculation')
else:
    idx_df['date'] = pd.to_datetime(idx_df['date'])
    # Weekly slice
    wk_df = idx_df[(idx_df['date']>=start_dt) & (idx_df['date']<=end_dt)]
    closes = idx_df.sort_values('date')
    first_ytd = closes.groupby('ticker').first()['close']
    last_ytd = closes.groupby('ticker').last()['close']
    ytd_ret = (last_ytd / first_ytd - 1) * 100
    # Weekly return: compare last close of week with first close of week
    weekly_group = wk_df.sort_values('date').groupby('ticker')
    week_first = weekly_group.first()['close']
    week_last = weekly_group.last()['close']
    weekly_ret = (week_last / week_first - 1) * 100
    # Weekly change (absolute points)
    weekly_change = week_last - week_first
    tbl = pd.DataFrame({
        'Weekly Return (%)': weekly_ret.round(2),
        'WeekLy Change': weekly_change.round(3),
        'YTD Return (%)': ytd_ret.round(2)
    })
    # Reorder with SP500 first
    order = [INDEX_TICKER] + [t for t in tbl.index if t != INDEX_TICKER]
    tbl = tbl.loc[order]
    # Produce markdown table
    md_lines = ['| Sector | Weekly Return (%) | WeekLy Change | YTD Return (%) |', '|--------|------------------:|--------------:|---------------:|']
    for tk, row in tbl.iterrows():
        md_lines.append(f"| {NAME_MAP.get(tk, tk)} | {row['Weekly Return (%)']:.2f} | {row['WeekLy Change']:.2f} | {row['YTD Return (%)']:.2f} |")
    markdown_table = '\n'.join(md_lines)
    from IPython.display import Markdown, display
    display(Markdown(markdown_table))


| Sector | Weekly Return (%) | WeekLy Change | YTD Return (%) |
|--------|------------------:|--------------:|---------------:|
| S&P 500 | -0.75 | -50.05 | 13.21 |
| Energy | 4.87 | 32.65 | 6.34 |
| Materials | -2.02 | -11.67 | 7.91 |
| Industrials | -0.29 | -3.73 | 16.22 |
| Cons Disc | -0.79 | -15.23 | 6.09 |
| Cons Staples | -0.21 | -1.81 | 2.09 |
| Health | -0.83 | -13.21 | -1.52 |
| Financials | -0.24 | -2.12 | 11.69 |
| Tech | -1.41 | -79.26 | 20.37 |
| Comm Services | -1.84 | -7.98 | 24.05 |
| Utilities | 1.89 | 8.16 | 13.71 |
| Real Estate | 0.56 | 1.47 | 4.06 |

In [ ]:
# 3. Mermaid radar for weekly High / Low / Close relative to prior week close (percent)
# We define each axis as: Close%, High%, Low% where % = (value / prior_week_close - 1)*100

# Need prior week last close
prior_start = start_week - timedelta(days=7)
prior_end = start_week - timedelta(days=1)
prior_query = {
    'ticker': {'$in': ALL_TICKERS},
    'date': {'$gte': datetime(prior_start.year, prior_start.month, prior_start.day), '$lte': datetime(prior_end.year, prior_end.month, prior_end.day)}
}
prior_rows = list(index_coll.find(prior_query, idx_proj))
prior_df = pd.DataFrame(prior_rows)
if prior_df.empty:
    print('Not enough prior week data for radar.')
else:
    prior_df['date'] = pd.to_datetime(prior_df['date'])
    prior_last = prior_df.sort_values('date').groupby('ticker').last()['close']
    wk_slice = idx_df[(idx_df['date']>=start_dt) & (idx_df['date']<=end_dt)]
    week_high = wk_slice.groupby('ticker')['high'].max()
    week_low = wk_slice.groupby('ticker')['low'].min()
    week_close = wk_slice.sort_values('date').groupby('ticker').last()['close']

    axes_tokens = []
    curve_tokens = []
    # Build axis list: we will have three axes per ticker? That would explode. Instead: axes = all tickers, curves = High/Low/Close? But ask wants one radar with high/low/close per sector.
    # We'll interpret as axes are tickers; each curve = High, Low, Close percentages.
    # Compute percentages relative to prior last close.
    pct_close = (week_close / prior_last) * 100
    pct_high = (week_high / prior_last) * 100
    pct_low = (week_low / prior_last) * 100

    # Limit to intersection tickers present in all three & prior
    common = pct_close.index.intersection(pct_high.index).intersection(pct_low.index)
    common = [t for t in common if t in ALL_TICKERS]
    if len(common) > 12:  # Mermaid readability threshold
        common = common[:12]

    for tk in common:
        axes_tokens.append(f"axis {tk.replace('^','').replace('-','')}['{NAME_MAP.get(tk, tk)}']")
    close_series = ', '.join(f"{pct_close[tk]:.2f}" for tk in common)
    high_series = ', '.join(f"{pct_high[tk]:.2f}" for tk in common)
    low_series = ', '.join(f"{pct_low[tk]:.2f}" for tk in common)
    curve_tokens.append(f"  curve ref['Reference']{{{','.join(['100.00']*len(common))}}}")
    curve_tokens.append(f"  curve h['High']{{{high_series}}}")
    curve_tokens.append(f"  curve c['Close']{{{close_series}}}")
    curve_tokens.append(f"  curve l['Low']{{{low_series}}}")

    max_val = float(pd.concat([pct_high.loc[common], pct_low.loc[common], pct_close.loc[common]]).max())
    min_val = float(pd.concat([pct_high.loc[common], pct_low.loc[common], pct_close.loc[common]]).min())
    pad = max(5, abs(max_val-min_val)*0.05)
    mmax = round(max_val + pad, 2)
    mmin = round(min_val - pad, 2)

    mermaid_lines = [
        '---',
        f'title: "Weekly Sector H/L/C % vs {start_dt} - {end_dt}"',
        'config:',
        '  themeVariables:',
        '    cScale0: "#D3D3D3"',
        '---',
        'radar-beta'
    ]
    mermaid_lines += ["  "+a for a in axes_tokens]
    mermaid_lines += curve_tokens
    mermaid_lines.append(f"  max {mmax}")
    mermaid_lines.append(f"  min {mmin}")


    mermaid_code = '\n'.join(mermaid_lines)
    print(mermaid_code)
    from IPython.display import Markdown, display
    display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


---
title: "Weekly Sector H/L/C % vs 2025-09-20 00:00:00 - 2025-09-26 00:00:00"
config:
  themeVariables:
    cScale0: "#D3D3D3"
---
radar-beta
  axis GSPE['Energy']
  axis SP50015['Materials']
  axis SP50020['Industrials']
  axis SP50025['Cons Disc']
  axis SP50030['Cons Staples']
  axis SP50035['Health']
  axis SP50040['Financials']
  axis SP50045['Tech']
  axis SP50050['Comm Services']
  axis SP50055['Utilities']
  axis SP50060['Real Estate']
  axis SPX['S&P 500']
  curve ref['Reference']{100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00}
  curve h['High']{105.70, 100.95, 101.21, 100.48, 100.14, 100.50, 100.77, 101.80, 100.23, 102.86, 101.31, 100.53}
  curve c['Close']{104.67, 97.98, 100.09, 98.78, 98.91, 99.13, 99.62, 100.31, 97.26, 102.82, 100.88, 99.69}
  curve l['Low']{99.29, 96.83, 98.83, 96.95, 98.44, 97.87, 98.49, 98.70, 96.21, 99.76, 99.50, 98.57}
  max 110.7
  min 91.21


```mermaid
---
title: "Weekly Sector H/L/C % vs 2025-09-20 00:00:00 - 2025-09-26 00:00:00"
config:
  themeVariables:
    cScale0: "#D3D3D3"
---
radar-beta
  axis GSPE['Energy']
  axis SP50015['Materials']
  axis SP50020['Industrials']
  axis SP50025['Cons Disc']
  axis SP50030['Cons Staples']
  axis SP50035['Health']
  axis SP50040['Financials']
  axis SP50045['Tech']
  axis SP50050['Comm Services']
  axis SP50055['Utilities']
  axis SP50060['Real Estate']
  axis SPX['S&P 500']
  curve ref['Reference']{100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00}
  curve h['High']{105.70, 100.95, 101.21, 100.48, 100.14, 100.50, 100.77, 101.80, 100.23, 102.86, 101.31, 100.53}
  curve c['Close']{104.67, 97.98, 100.09, 98.78, 98.91, 99.13, 99.62, 100.31, 97.26, 102.82, 100.88, 99.69}
  curve l['Low']{99.29, 96.83, 98.83, 96.95, 98.44, 97.87, 98.49, 98.70, 96.21, 99.76, 99.50, 98.57}
  max 110.7
  min 91.21
```